In [1]:
# Setup cell
import sys, os, time, json
sys.path.append(os.path.abspath(''))  # project root
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pprint import pprint

In [19]:
import sys
print(sys.executable)

C:\Python313\python.exe


In [21]:
!C:\Python313\python.exe -m pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_learn-1.7.2-cp313-cp313-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.16.3-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.7.2-cp313-cp313-win_amd64.whl (8.7 MB)
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
Using cached scipy-1.16.3-cp313-cp313-win_amd64.whl (38.5 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   -------

In [22]:
# local utilities / loaders
from my_ml_lib.datasets._loaders import load_fashion_mnist
from my_ml_lib.model_selection._split import train_test_split
from my_ml_lib.preprocessing._data import StandardScaler
from my_ml_lib.preprocessing._polynomial import PolynomialFeatures
from my_ml_lib.preprocessing._gaussian import GaussianBasisFeatures
from utils.io_utils import save_model

In [23]:
# linear / classification models
from my_ml_lib.linear_models.classification._logistic import LogisticRegression

# MLP/autograd modules
from my_ml_lib.nn.modules.containers import Sequential
from my_ml_lib.nn.modules.linear import Linear
from my_ml_lib.nn.modules.activations import ReLU, Sigmoid
from my_ml_lib.nn.optim import SGD as NN_SGD
from my_ml_lib.nn.losses import CrossEntropyLoss
from my_ml_lib.nn.autograd import Value

In [24]:
# make results folder
os.makedirs("results/p5_capstone_showdown", exist_ok=True)

In [25]:
RNG = 0
np.random.seed(RNG)

In [26]:
#Train Val split: 80% train and 20% validation
X_all, y_all = load_fashion_mnist()
X_train, X_val, y_train, y_val = train_test_split(X_all, y_all, test_size=0.2, shuffle=True, random_state=RNG)

print("Shapes: X_train", X_train.shape, "X_val", X_val.shape)
# Optionally standardize (recommended for linear models)
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s = scaler.transform(X_val)

Shapes: X_train (48000, 784) X_val (12000, 784)


#One-VS-Rest Logistic (OVR)

In [ ]:
# Model 1: OvR Logistic
from copy import deepcopy
from sklearn.metrics import confusion_matrix, accuracy_score  # optional to display confusion

alpha = 1e-3
max_iter = 200

print("Training OvR logistic: alpha=", alpha, "max_iter=", max_iter)
classes = np.unique(y_train)
ovr_models = {}
train_scores = []
val_scores = []

# Train one classifier per class (binary)
for c in classes:
    y_bin = (y_train == c).astype(int)
    clf = LogisticRegression(alpha=alpha, max_iter=max_iter, tol=1e-6, verbose=0)
    clf.fit(X_train_s, y_bin)
    ovr_models[int(c)] = deepcopy(clf)

# helper to get class probabilities per sample (score of class=1)
def ovr_predict_proba(models, X):
    ps = []
    for c in classes:
        ps.append(models[int(c)].predict_proba(X)[:,1])
    return np.vstack(ps).T  # shape (n, C)

# evaluate
yhat_train = np.argmax(ovr_predict_proba(ovr_models, X_train_s), axis=1)
yhat_val = np.argmax(ovr_predict_proba(ovr_models, X_val_s), axis=1)
train_acc = accuracy_score(y_train, yhat_train)
val_acc = accuracy_score(y_val, yhat_val)
print(f"OvR Train acc: {train_acc:.4f}  Val acc: {val_acc:.4f}")

# save model and confusion matrix
save_model(ovr_models, model_name=f"ovr_best_{datetime.now().strftime('%Y%m%d_%H%M%S')}")
cm = confusion_matrix(y_val, yhat_val)
plt.figure(figsize=(6,4))
plt.imshow(cm, cmap='Blues')
plt.title(f"OvR Confusion Matrix (val) Acc={val_acc:.4f}")
plt.colorbar()
plt.tight_layout()
plt.savefig(f"results/p5_showdown/ovr_confusion_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png", dpi=200)
plt.show()

Training OvR logistic: alpha= 0.001 max_iter= 200
